### SJM-BL Simulation Study with Parallel Monte Carlo

This code runs multiple Monte Carlo simulations of the 1-state, 2-state, and 3-state processes, computes performance metrics for each run, and then uses a Wilcoxon test to compare SJM-BL against all other strategies.

#### 1.0 Loading packages

In [ ]:
# 1.0 Loading packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
from joblib import Parallel, delayed
import multiprocessing

# Hidden Markov Model utilities
from hmmlearn.hmm import GaussianHMM
from sklearn.cluster import KMeans

# PyPortfolioOpt
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

# Sparse Jump Model utilities
from jumpmodels.sparse_jump import SparseJumpModel
from jumpmodels.preprocess import StandardScalerPD, DataClipperStd


### 2.0 Data Simulation

#### 2.1 Simulating the 1-state data (Gaussian, not Student-t)
- All assets share the same mean/vol and correlation structure.

In [115]:
ASSETS = ["Value", "Growth", "LowVol", "Size", "Momentum", "Quality"]
N_ASSETS = len(ASSETS)
CONST_RET = 0.000461   # Hypothetical daily return used
RISK_FREE_RATE = 0
TRANSACTION_COST = 0.0005  # Cost per dollar transacted
BL_TAU = 0.1  # Black-Litterman tau parameter

In [116]:
def simulate_1state_data(num_days, seed=None):
    """
    Single-state returns (Gaussian). 
    Kept correlation and variance, removed Student-t scaling.
    """
    local_rng = np.random.default_rng(seed)
    mu = 0.000461
    sig = 0.008388

    corr = np.full((N_ASSETS, N_ASSETS), 0.185)
    np.fill_diagonal(corr, 1.0)
    cov = np.outer(np.full(N_ASSETS, sig), np.full(N_ASSETS, sig)) * corr

    # Gaussian draws
    rets = local_rng.multivariate_normal(mean=np.full(N_ASSETS, mu), 
                                         cov=cov, 
                                         size=num_days)
    return pd.DataFrame(rets, columns=ASSETS)

#### 2.2 Simulating 2-state data

This function simulates a 2-state HMM (bull/bear) with state‐dependent Student‑t returns.

In [117]:
def simulate_2state_data(num_days, seed=None):
    """
    2-state HMM-like simulation (Gaussian) **per asset**.

    Each of the 6 assets has its own 2-state Markov chain, so
    they do NOT necessarily move to bull/bear at the same time.
    We still allow cross-asset correlation on each day based on 
    the current state's volatility for each asset.
    """
    local_rng = np.random.default_rng(seed)

    # Transition matrix for each asset's 2-state chain
    transmat = np.array([
        [0.9976, 0.0024],
        [0.0232, 0.9768]
    ])

    # State-specific means and std dev for all assets (shared here for simplicity)
    mu_dict = {0: 0.0006, 1: -0.000881}
    sig_dict = {0: 0.00757, 1: 0.0163}

    # Correlation matrix (off-diagonal = 0.185, diag = 1.0)
    # We'll apply scaling day-by-day based on each asset's current state.
    base_corr = np.full((N_ASSETS, N_ASSETS), 0.185)
    np.fill_diagonal(base_corr, 1.0)

    # For each asset, we generate its own Markov chain of states
    all_states = np.zeros((num_days, N_ASSETS), dtype=int)
    for i in range(N_ASSETS):
        # Single chain for asset i
        s = np.zeros(num_days, dtype=int)
        s[0] = local_rng.integers(2)
        for t in range(1, num_days):
            s[t] = local_rng.choice(2, p=transmat[s[t - 1]])
        all_states[:, i] = s

    # Now generate returns day-by-day
    rets = np.zeros((num_days, N_ASSETS))
    for t in range(num_days):
        # Build mean vector & stdev vector for each asset 
        mu_vec = np.zeros(N_ASSETS)
        sig_vec = np.zeros(N_ASSETS)
        for i in range(N_ASSETS):
            curr_state = all_states[t, i]
            mu_vec[i] = mu_dict[curr_state]
            sig_vec[i] = sig_dict[curr_state]

        # Build the day-t covariance = diag(sig_vec) * base_corr * diag(sig_vec)
        cov_t = np.outer(sig_vec, sig_vec) * base_corr

        # Draw one 6-dimensional sample for day t
        rets[t] = local_rng.multivariate_normal(mean=mu_vec, cov=cov_t)

    # Return DF plus the full (T x N_ASSETS) state array if you need it
    return pd.DataFrame(rets, columns=ASSETS), all_states

#### 2.3 Simulating 3-state data

We are simulating 6 fictional assets which are representing the 6 factors in our framework

In [118]:
def simulate_3state_data(num_days, seed=None):
    """
    3-state HMM-like simulation (Gaussian) **per asset**.

    Each asset has its own 3-state Markov chain. 
    They won't all jump to the same state at once. 
    We apply each asset's state's volatility into 
    a correlation matrix on each day.
    """
    local_rng = np.random.default_rng(seed)

    # Common transition matrix for each asset's 3-state chain
    transmat = np.array([
        [0.9989, 0.0004, 0.0007],
        [0.0089, 0.9904, 0.0007],
        [0.0089, 0.0004, 0.9907]
    ])

    # State-specific means and stdev for each of the 3 states
    mu_list = [0.0008, 0.0, -0.003586]
    sig_list = [0.0070, 0.0050, 0.01897]

    # Base correlation matrix
    base_corr = np.full((N_ASSETS, N_ASSETS), 0.185)
    np.fill_diagonal(base_corr, 1.0)

    # Generate a separate Markov chain for each of the 6 assets
    all_states = np.zeros((num_days, N_ASSETS), dtype=int)
    for i in range(N_ASSETS):
        s = np.zeros(num_days, dtype=int)
        s[0] = local_rng.integers(3)
        for t in range(1, num_days):
            s[t] = local_rng.choice(3, p=transmat[s[t - 1]])
        all_states[:, i] = s

    # Generate returns day-by-day
    rets = np.zeros((num_days, N_ASSETS))
    for t in range(num_days):
        mu_vec = np.zeros(N_ASSETS)
        sig_vec = np.zeros(N_ASSETS)
        for i in range(N_ASSETS):
            st_i = all_states[t, i]
            mu_vec[i] = mu_list[st_i]
            sig_vec[i] = sig_list[st_i]

        # Cov for day t
        cov_t = np.outer(sig_vec, sig_vec) * base_corr

        # Sample a 6D Gaussian
        rets[t] = local_rng.multivariate_normal(mean=mu_vec, cov=cov_t)

    return pd.DataFrame(rets, columns=ASSETS), all_states

### 3.0 Training Regime Models

#### 3.1 Training HMM using kmeans clustering initialization

In [119]:
def run_mle(observations, n_components=2, init_type='default', seed=None):
    model = GaussianHMM(n_components=n_components, covariance_type='diag',
                        n_iter=100, random_state=seed)

    if init_type == 'default':
        model.startprob_ = np.array([1.0, 0.0])
        model.transmat_ = np.array([[0.9, 0.1],
                                    [0.1, 0.9]])
        model.means_ = np.zeros((n_components, observations.shape[1]))
        model.covars_ = np.full((n_components, observations.shape[1]), 1e-10)
        model.init_params = ''  # stops re-initializing
    elif init_type == 'kmeans':
        km = KMeans(n_clusters=n_components, n_init=10, random_state=seed)
        labels = km.fit_predict(observations)
        means, covars = [], []
        for i in range(n_components):
            obs_i = observations[labels == i]
            means.append(np.mean(obs_i, axis=0))
            covars.append(np.var(obs_i, axis=0) + 1e-10)
        model.startprob_ = np.ones(n_components) / n_components
        model.transmat_ = np.ones((n_components, n_components)) / n_components
        model.means_ = np.array(means)
        model.covars_ = np.array(covars)
        model.init_params = ''

    model.fit(observations)
    pred_states = model.predict(observations)
    return model, pred_states

In [120]:
def run_mle_default(observations, seed=None):
    return run_mle(observations, init_type='default', seed=seed)

def run_mle_kmeans(observations, seed=None):
    return run_mle(observations, init_type='kmeans', seed=seed)

def train_hmm_single_asset_default(series, n_components=2, random_state=42):
    X = series.values.reshape(-1, 1)
    model, _ = run_mle_default(X, seed=random_state)
    return model

def train_hmm_single_asset_kmeans(series, n_components=2, random_state=42):
    X = series.values.reshape(-1, 1)
    model, _ = run_mle_kmeans(X, seed=random_state)
    return model

#### 3.2 Training Sparse Jump model with max_feats=9 and lambda=90
##### 3.2.1 Defining feature selection framework

In [121]:
def compute_sjm_features(factor_ser: pd.Series) -> pd.DataFrame:
    """
    Build strictly backward-looking features for a single factor 'factor_ser'.
    No external market data is used.
    
    Returns a DataFrame with 12 columns:
      - EWMA of factor's daily returns (8/21/63)
      - RSI (8/21/63)
      - Stochastic Oscillator (8/21/63)
      - MACD (8,21) and (21,63)
      - Log-Downside Deviation (21)
    """
    import numpy as np
    
    factor_price = 100.0 * (1.0 + factor_ser).cumprod()

    def ewma_return(returns, halflife):
        return returns.ewm(halflife=halflife).mean()

    def compute_rsi(price, window):
        delta = price.diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.rolling(window).mean()
        avg_loss = loss.rolling(window).mean()
        rs = avg_gain / avg_loss.replace(0, np.nan)
        return 100.0 - (100.0 / (1.0 + rs))

    def compute_stoch(price, window):
        rolling_min = price.rolling(window).min()
        rolling_max = price.rolling(window).max()
        return 100.0 * (price - rolling_min) / (rolling_max - rolling_min)

    def compute_macd(price, fast, slow):
        ema_fast = price.ewm(halflife=fast).mean()
        ema_slow = price.ewm(halflife=slow).mean()
        return ema_fast - ema_slow

    def compute_downside_dev_log(returns, window):
        def _downside(subarray):
            negatives = np.where(subarray < 0, subarray, 0.0)
            return np.sqrt((negatives**2).mean())
        dd = returns.rolling(window).apply(_downside, raw=True)
        return np.log(dd.replace(0, np.nan))

    feats = {}
    # 3 EWMAs
    for hl in [8, 21, 63]:
        feats[f"FactorRet_EWMA_{hl}"] = ewma_return(factor_ser, hl)
    # 3 RSI
    for w in [8, 21, 63]:
        feats[f"RSI_{w}"] = compute_rsi(factor_price, w)
    # 3 Stoch
    for w in [8, 21, 63]:
        feats[f"Stoch%K_{w}"] = compute_stoch(factor_price, w)
    # MACD (8,21) and (21,63)
    feats["MACD_8_21"] = compute_macd(factor_price, 8, 21)
    feats["MACD_21_63"] = compute_macd(factor_price, 21, 63)
    # Log-Downside Deviation (21)
    feats["DownsideDev_log_21"] = compute_downside_dev_log(factor_ser, 21)
    
    return pd.DataFrame(feats)

In [122]:
def train_sjm_single_asset(series, n_components=2, max_feats=12, lam=90, random_state=42):
    """
    Sparse Jump Model training for a single asset using strictly backward-looking features.
    Returns (sjm, clipper, scaler) so the same transformations can be applied on test data.
    """
    # 1) Build new backward-looking features
    feats_df = compute_sjm_features(series)
    
    # 2) Replace infinities and fill NaNs (e.g., the first rolling window days, etc.)
    feats_df = feats_df.replace([np.inf, -np.inf], np.nan)
    feats_df = feats_df.fillna(0.0)  # or fillna(method='ffill'), etc.

    # 3) Clip & Scale
    clipper = DataClipperStd(mul=3.0)
    scaler = StandardScalerPD()

    X_clipped = clipper.fit_transform(feats_df)
    X_scaled = scaler.fit_transform(X_clipped)
    X_arr = X_scaled.values

    # 4) Fit SJM
    sjm = SparseJumpModel(
        n_components=n_components,
        max_feats=max_feats,
        jump_penalty=lam,
        cont=False,
        max_iter=20,
        random_state=random_state
    )
    sjm.fit(X_arr)

    return sjm, clipper, scaler

### 4.0 Allocation simulation

#### 4.1 Allocation workhorse functions
In this code we create the in which we fit the following models (each done in a seperate for loop such that we can store the relevant data such as return, weights, etc. in seperate dfs):
1. Equal weigted
2. Inverse volatility weighted
3. Mean-Variance-Optimal static portfolio
4. Hidden Markov Model Black Litterman where infered states are the identified regimes
5. Sparse Jump Model Black Litterman where infered states are the identified regimes

In [123]:
def backtest_portfolio(returns, weights):
    """
    Backtest a static portfolio with single allocation.
    Includes initial transaction cost for entering the position.
    """
    T = len(returns)
    portfolio_vals = np.zeros(T)
    # Subtract initial transaction cost on day 0
    cost_init = 1.0 * np.sum(np.abs(weights)) * TRANSACTION_COST
    portfolio_vals[0] = 1.0 - cost_init  # start net of initial cost

    for t in range(T - 1):
        ret_t = returns.iloc[t].values
        # No daily rebalancing, so no daily cost
        portfolio_vals[t + 1] = portfolio_vals[t] * (1.0 + np.dot(weights, ret_t))

    return portfolio_vals

def equal_weight_allocation(n_assets):
    """Equal weight."""
    return np.ones(n_assets) / n_assets

def inverse_vol_weights(returns):
    """Inverse volatility weights."""
    stds = returns.std(axis=0).values + 1e-12
    w = 1.0 / stds
    return w / w.sum()

def static_mvo_allocation(returns):
    """
    Simple static mean-variance optimization using PyPortfolioOpt.
    """
    from pypfopt import expected_returns
    # We keep a constant mu for demonstration
    mu = pd.Series(CONST_RET, index=returns.columns)
    cov = risk_models.sample_cov(returns)

    ef = EfficientFrontier(mu, cov, weight_bounds=(0, 1), solver="OSQP")
    ef_weights = ef.max_sharpe(risk_free_rate=RISK_FREE_RATE)
    return ef.clean_weights()

def black_litterman_allocation(view_vector, prior_cov):
    """
    Basic Black-Litterman with absolute views on each asset.
    """
    pi = pd.Series(CONST_RET, index=prior_cov.columns)
    viewdict = {asset: v for asset, v in zip(ASSETS, view_vector)}

    bl = BlackLittermanModel(
        cov_matrix=prior_cov,
        pi=pi,
        absolute_views=viewdict,
        tau=BL_TAU,
        risk_aversion=1
    )
    bl_rets = bl.bl_returns()  
    bl_cov = bl.bl_cov()

    ef = EfficientFrontier(bl_rets, bl_cov, weight_bounds=(0,1), solver="OSQP")
    if max(bl_rets) <= RISK_FREE_RATE:
        ef_weights = ef.min_volatility()
    else:
        ef_weights = ef.max_sharpe(risk_free_rate=RISK_FREE_RATE)

    clean_weights = ef.clean_weights()
    w_array = np.array([clean_weights[a] for a in prior_cov.columns])
    return w_array

### 5.0 Performance metric evaluation:
Here we divide the performance metric into. We assume 250 data points to be 1 year off trading:
1. Return-Based Metrics 

Annualized Return: Average return per year. 

Cumulative Return: Total portfolio growth over time. 

2. Risk-Based Metrics 

Volatility: Standard deviation of returns. 

Downside Deviation: Measures negative return fluctuations. 

Max Drawdown (MDD): Largest portfolio decline from peak to trough. 

3. Risk-Adjusted Metrics 

Sharpe Ratio: Return per unit of total risk. 

Sortino Ratio: Return per unit of downside risk. 

Calmar Ratio: Return relative to max drawdown. 

4. Portfolio Stability & Adaptation 

Turnover Rate: Measures frequency of asset reallocation. 


We further split the performance three seperate tables with 1-state process, 2-state process, 3-state process




In [124]:
def compute_performance_metrics(portfolio_vals, weight_history=None, annual_factor=250):
    """
    Calculate performance stats on the final portfolio_vals series.
    """
    pv = np.asarray(portfolio_vals)
    rets = np.diff(pv) / pv[:-1]

    ann_ret = rets.mean() * annual_factor
    cum_ret = pv[-1] / pv[0] - 1
    ann_vol = rets.std() * np.sqrt(annual_factor)

    negative_rets = rets[rets < 0]
    ddev = negative_rets.std() * np.sqrt(annual_factor) if len(negative_rets) > 0 else 0.0
    max_dd = (pv / np.maximum.accumulate(pv) - 1).min()

    sharpe = ann_ret / (ann_vol + 1e-12)
    sortino = ann_ret / ddev if ddev > 1e-12 else np.nan
    calmar = ann_ret / abs(max_dd) if max_dd < 0 else np.nan

    if weight_history is not None and len(weight_history) > 1:
        turnovers = []
        for t in range(1, len(weight_history)):
            turnovers.append(np.sum(np.abs(weight_history[t] - weight_history[t - 1])))
        avg_turnover = np.mean(turnovers)
    else:
        avg_turnover = 0.0

    return {
        "Annualized Return": ann_ret,  # net of costs included in PV path
        "Cumulative Return": cum_ret,
        "Volatility": ann_vol,
        "Downside Deviation": ddev,
        "Max Drawdown": max_dd,
        "Sharpe Ratio": sharpe,
        "Sortino Ratio": sortino,
        "Calmar Ratio": calmar,
        "Turnover Rate": avg_turnover,
    }

#### 6.0 Helper function

In [125]:
def get_regime_means_single_asset(asset_series, regime_assignments):
    """
    Returns {regime_label: mean_return_in_that_regime}.
    """
    unique_states = np.unique(regime_assignments)
    regime_means = {}
    for s in unique_states:
        regime_means[s] = asset_series[regime_assignments == s].mean()
    return regime_means

#### 7.0 Regime Based Asset Allocaiton

In [126]:
def regime_based_bl_backtest(df_test, states_test, regime_means_list, prior_cov, train_means_per_asset):
    """
    Daily rebalancing. On each day t:
      1) Identify regime per asset -> set BL views
      2) Compute new BL weights
      3) Deduct transaction cost on rebal
    """
    T_test = len(df_test)
    portfolio_vals = np.zeros(T_test)
    portfolio_vals[0] = 1.0

    weight_history = np.zeros((T_test, N_ASSETS))
    w_prev = equal_weight_allocation(N_ASSETS)
    weight_history[0] = w_prev

    for t in range(T_test - 1):
        # Construct BL views from current states
        view_vector = []
        for i in range(N_ASSETS):
            current_regime = states_test[t, i]
            # if regime unknown, fallback to train mean
            view_val = regime_means_list[i].get(current_regime, train_means_per_asset[i])
            view_vector.append(view_val)

        # Rebalance
        w_bl = black_litterman_allocation(view_vector, prior_cov)

        # Calculate daily return
        ret_t = df_test.iloc[t].values
        gross_growth = portfolio_vals[t] * (1.0 + np.dot(w_prev, ret_t))

        # Calculate transaction cost (portfolio-based)
        traded_fraction = np.sum(np.abs(w_bl - w_prev))
        cost = gross_growth * traded_fraction * TRANSACTION_COST

        # Net portfolio after cost
        portfolio_vals[t + 1] = gross_growth - cost
        weight_history[t + 1] = w_bl
        w_prev = w_bl

    return portfolio_vals, weight_history

#### 8.0 Wrapper to run full allocation

In [127]:
def run_allocation(df, lam_sjm=90):
    """
    Splits df into train/test. Trains HMM and SJM per asset. 
    Then runs:
      1) Equal Weight (static)
      2) Inverse Vol (static)
      3) Static MVO
      4) HMM-BL (default)
      5) HMM-BL (kmeans)
      6) SJM-BL
    Returns performance dict for each strategy.
    """
    split_idx = int(len(df) * 0.8)
    df_train = df.iloc[:split_idx]
    df_test = df.iloc[split_idx:]
    prior_cov = df_train.cov()

    hmm_models_default = []
    hmm_models_kmeans = []
    sjm_models = []
    sjm_clippers = []    # Store each asset's clipper
    sjm_scalers = []     # Store each asset's scaler

    hmm_states_default_train = np.zeros((split_idx, N_ASSETS), dtype=int)
    hmm_states_kmeans_train = np.zeros((split_idx, N_ASSETS), dtype=int)
    sjm_states_train = np.zeros((split_idx, N_ASSETS), dtype=int)

    # ---- 1) Train per-asset models ----
    for i, asset in enumerate(ASSETS):
        series_train = df_train[asset]

        # HMM (default)
        hmm_mod_default = train_hmm_single_asset_default(series_train)
        states_def = hmm_mod_default.predict(series_train.values.reshape(-1, 1))
        hmm_models_default.append(hmm_mod_default)
        hmm_states_default_train[:, i] = states_def

        # HMM (kmeans)
        hmm_mod_kmeans = train_hmm_single_asset_kmeans(series_train)
        states_km = hmm_mod_kmeans.predict(series_train.values.reshape(-1, 1))
        hmm_models_kmeans.append(hmm_mod_kmeans)
        hmm_states_kmeans_train[:, i] = states_km

        # SJM
        sjm_mod, sjm_clip, sjm_scale = train_sjm_single_asset(
            series_train,
            n_components=2,
            max_feats=12,
            lam=lam_sjm
        )
        # We also need to get the training set states:
        feats_train = compute_sjm_features(series_train)
        feats_train = feats_train.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        X_train_clipped = sjm_clip.transform(feats_train)
        X_train_scaled = sjm_scale.transform(X_train_clipped)
        states_sjm = sjm_mod.predict(X_train_scaled)

        sjm_models.append(sjm_mod)
        sjm_clippers.append(sjm_clip)
        sjm_scalers.append(sjm_scale)
        sjm_states_train[:, i] = states_sjm

    # ---- 2) In-sample regime means ----
    hmm_regime_means_default = []
    hmm_regime_means_kmeans = []
    sjm_regime_means = []
    train_means_per_asset = []
    for i in range(N_ASSETS):
        asset_train = df_train.iloc[:, i]
        train_means_per_asset.append(asset_train.mean())

        hmm_regime_means_default.append(
            get_regime_means_single_asset(asset_train, hmm_states_default_train[:, i])
        )
        hmm_regime_means_kmeans.append(
            get_regime_means_single_asset(asset_train, hmm_states_kmeans_train[:, i])
        )
        sjm_regime_means.append(
            get_regime_means_single_asset(asset_train, sjm_states_train[:, i])
        )

    # ---- 3) Predict states on test set ----
    T_test = len(df_test)
    hmm_states_default_test = np.zeros((T_test, N_ASSETS), dtype=int)
    hmm_states_kmeans_test = np.zeros((T_test, N_ASSETS), dtype=int)
    sjm_states_test = np.zeros((T_test, N_ASSETS), dtype=int)

    for i, asset in enumerate(ASSETS):
        # HMM test states
        asset_series_test = df_test[asset].values.reshape(-1, 1)
        hmm_states_default_test[:, i] = hmm_models_default[i].predict(asset_series_test)
        hmm_states_kmeans_test[:, i] = hmm_models_kmeans[i].predict(asset_series_test)

        # SJM test states
        feats_test = compute_sjm_features(df_test[asset])
        feats_test = feats_test.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        X_test_clipped = sjm_clippers[i].transform(feats_test)
        X_test_scaled = sjm_scalers[i].transform(X_test_clipped)
        sjm_states_test[:, i] = sjm_models[i].predict(X_test_scaled)

    # ---- 4) Build & Evaluate Strategies ----
    w_ew = equal_weight_allocation(N_ASSETS)
    pv_ew = backtest_portfolio(df_test, w_ew)
    w_hist_ew = np.tile(w_ew, (T_test, 1))

    w_iv = inverse_vol_weights(df_test)
    pv_iv = backtest_portfolio(df_test, w_iv)
    w_hist_iv = np.tile(w_iv, (T_test, 1))

    w_mvo_dict = static_mvo_allocation(df_train)
    w_mvo_arr = np.array([w_mvo_dict[a] for a in ASSETS])
    pv_mvo = backtest_portfolio(df_test, w_mvo_arr)
    w_hist_mvo = np.tile(w_mvo_arr, (T_test, 1))

    pv_hmmbl_default, w_hmmbl_default = regime_based_bl_backtest(
        df_test, hmm_states_default_test,
        hmm_regime_means_default, prior_cov, train_means_per_asset
    )
    pv_hmmbl_kmeans, w_hmmbl_kmeans = regime_based_bl_backtest(
        df_test, hmm_states_kmeans_test,
        hmm_regime_means_kmeans, prior_cov, train_means_per_asset
    )
    pv_sjmbl, w_sjmbl = regime_based_bl_backtest(
        df_test, sjm_states_test,
        sjm_regime_means, prior_cov, train_means_per_asset
    )

    # ---- 5) Collect Performance ----
    perf = {
        "EW": compute_performance_metrics(pv_ew, w_hist_ew),
        "IV": compute_performance_metrics(pv_iv, w_hist_iv),
        "MVO": compute_performance_metrics(pv_mvo, w_hist_mvo),
        "HMM-BL-Default": compute_performance_metrics(pv_hmmbl_default, w_hmmbl_default),
        "HMM-BL-KMeans": compute_performance_metrics(pv_hmmbl_kmeans, w_hmmbl_kmeans),
        "SJM-BL": compute_performance_metrics(pv_sjmbl, w_sjmbl)
    }

    return perf

#### 9.0 Full scenario: 1-state, 2-state, 3-state runs



In [128]:
def run_scenario_123(T_sim=1000, seed1=None, seed2=None, seed3=None):
    """
    Simulate & run 1-state, 2-state, 3-state data sets.
    """
    df1_full = simulate_1state_data(T_sim, seed=seed1)
    perf_1 = run_allocation(df1_full)

    df2_full, _ = simulate_2state_data(T_sim, seed=seed2)
    perf_2 = run_allocation(df2_full)

    df3_full, _ = simulate_3state_data(T_sim, seed=seed3)
    perf_3 = run_allocation(df3_full)

    return {
        "1state": perf_1,
        "2state": perf_2,
        "3state": perf_3
    }

def single_monte_carlo_run(run_id, T_sim=1000):
    """
    A single replication with 3 scenarios (1-,2-,3-state).
    """
    print(f"Running simulation {run_id}...")
    seed_for_1state = run_id * 1000 + 11
    seed_for_2state = run_id * 1000 + 22
    seed_for_3state = run_id * 1000 + 33

    results = run_scenario_123(
        T_sim=T_sim,
        seed1=seed_for_1state,
        seed2=seed_for_2state,
        seed3=seed_for_3state
    )
    return results

def run_monte_carlo_study(n_runs=10, T_sim=1000):
    """
    Runs multiple replications in parallel. Then does Wilcoxon test on Sharpe Ratios.
    """
    n_cores = multiprocessing.cpu_count()
    print(f"detected {n_cores} cores")

    all_results = Parallel(n_jobs=n_cores)(
        delayed(single_monte_carlo_run)(i+1, T_sim) for i in range(n_runs)
    )

    # Strategies
    strategies = ["EW", "IV", "MVO", "HMM-BL-Default", "HMM-BL-KMeans", "SJM-BL"]
    scenarios = ["1state", "2state", "3state"]

    # Sharpe data for Wilcoxon
    sharpe_data = {sc: {st: [] for st in strategies} for sc in scenarios}

    # Store entire distribution of metrics
    all_metrics = {}
    for sc in scenarios:
        all_metrics[sc] = {}
        for st in strategies:
            all_metrics[sc][st] = {
                "Annualized Return": [],
                "Cumulative Return": [],
                "Volatility": [],
                "Downside Deviation": [],
                "Max Drawdown": [],
                "Sharpe Ratio": [],
                "Sortino Ratio": [],
                "Calmar Ratio": [],
                "Turnover Rate": [],
            }

    # Collect metrics
    for run_res in all_results:
        for sc in scenarios:
            for st in strategies:
                metrics_dict = run_res[sc][st]
                sharpe_data[sc][st].append(metrics_dict["Sharpe Ratio"])
                for mkey in all_metrics[sc][st]:
                    all_metrics[sc][st][mkey].append(metrics_dict[mkey])

    # Wilcoxon test: SJM-BL vs others (Sharpe)
    print("\n==== Wilcoxon Tests (SJM-BL vs. others) ====")
    wilcoxon_rows = []
    for sc in scenarios:
        sjm_sharpes = sharpe_data[sc]["SJM-BL"]
        for st in strategies:
            if st == "SJM-BL":
                continue
            other_sharpes = sharpe_data[sc][st]
            stat, pval = wilcoxon(sjm_sharpes, other_sharpes, alternative='two-sided')
            print(f"{sc} | SJM-BL vs {st}: stat={stat:.4f}, p={pval:.4g}")
            wilcoxon_rows.append({
                "Scenario": sc,
                "Comparison": f"SJM-BL vs {st}",
                "Statistic": stat,
                "p-value": pval
            })

    df_wilcoxon = pd.DataFrame(wilcoxon_rows)
    print("\nWilcoxon Results Table:")
    print(df_wilcoxon.to_string(index=False))

    # Print average metrics
    print("\n==== Average Performance Metrics (across runs) ====")
    for sc in scenarios:
        rows = []
        for st in strategies:
            metric_means = {}
            for mkey, vals in all_metrics[sc][st].items():
                metric_means[mkey] = np.mean(vals)
            row = {"Strategy": st}
            row.update(metric_means)
            rows.append(row)
        df_avg = pd.DataFrame(rows)
        df_avg.set_index("Strategy", inplace=True)
        print(f"\n--- {sc.upper()} ---")
        print(df_avg.to_string())

    return sharpe_data, all_metrics, all_results, df_wilcoxon

### 10.0 Main execution: Run simulation and output performance metrics

In [129]:
if __name__ == "__main__":
    # Example: run 5 replications
    n_simulations = 24
    T_sim = 5000

    # Run parallel simulation
    sharpe_data, all_metrics, all_runs, df_wilcoxon = run_monte_carlo_study(
        n_runs=n_simulations,
        T_sim=T_sim
    )

detected 8 cores


Running simulation 4...Running simulation 7...

Running simulation 5...
Running simulation 1...
Running simulation 8...
Running simulation 2...
Running simulation 3...
Running simulation 6...


Model is not converging.  Current: 13399.97468682194 is not greater than 13400.026712611634. Delta is -0.05202578969328897
Model is not converging.  Current: 13461.444512960772 is not greater than 13461.462574048224. Delta is -0.018061087452224456
Model is not converging.  Current: 13430.843406489204 is not greater than 13430.876458913983. Delta is -0.033052424778361456
Model is not converging.  Current: 13417.123647420682 is not greater than 13417.135908083997. Delta is -0.012260663315828424
Model is not converging.  Current: 13477.861014403064 is not greater than 13477.897984230327. Delta is -0.03696982726251008
Model is not converging.  Current: 13384.939811275972 is not greater than 13384.963567917232. Delta is -0.023756641259751632
Model is not converging.  Current: 13458.44335717629 is not greater than 13458.466881097735. Delta is -0.02352392144530313
Model is not converging.  Current: 13447.338113765592 is not greater than 13447.353301179308. Delta is -0.015187413715466391
Model

OptimizationError: ('Please check your objectives/constraints or use a different solver.', 'Please check your objectives/constraints or use a different solver.', 'Solver status: infeasible_inaccurate')